In [ ]:
import cv2
import pickle
import numpy as np
import os
import requests
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.applications.efficientnet import EfficientNetB3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from numpy.linalg import norm


In [ ]:

# Load EfficientNetB3 và hàm trích xuất đặc trưng
def load_model():
    """Tải mô hình EfficientNetB3"""
    base_model = EfficientNetB3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = tf.keras.Sequential([
        base_model,
        GlobalMaxPooling2D()
    ])
    return model

def extract_features(img_path, model):
    """Trích xuất đặc trưng ảnh từ EfficientNetB3"""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)

    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)  # Chuẩn hóa L2
    return normalized_result
# Hàm tạo ảnh mới bằng Data Augmentation
def augment_image(img_path):
    """Tạo ảnh mới bằng Data Augmentation"""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2]
    )

    aug_iter = datagen.flow(img_array, batch_size=1)
    aug_img = next(aug_iter)[0].astype(np.uint8)
    return aug_img
# Hàm tạo ảnh mới bằng Data Augmentation
def augment_image(img_path):
    """Tạo ảnh mới bằng Data Augmentation"""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2]
    )

    aug_iter = datagen.flow(img_array, batch_size=1)
    aug_img = next(aug_iter)[0].astype(np.uint8)
    return aug_img
def save_data(features_dict):
    """Lưu embeddings và filenames"""
    filenames = np.array(list(features_dict.keys()))
    feature_list = np.array(list(features_dict.values()))

    if len(feature_list) > 5:
        n_components = min(512, len(feature_list))
        pca = PCA(n_components=n_components)
        reduced_features = pca.fit_transform(feature_list)
        pickle.dump(pca, open("pca.pkl", "wb"))
        print(f"✅ Áp dụng PCA với {n_components} thành phần!")

    pickle.dump(features_dict, open("features.pkl", "wb"))
    pickle.dump(filenames, open("filenames.pkl", "wb"))

    # Tạo mô hình Nearest Neighbors
    nn_model = NearestNeighbors(n_neighbors=4, metric="cosine")
    nn_model.fit(feature_list)
    pickle.dump(nn_model, open("nn_model.pkl", "wb"))

    print("✅ Đã lưu embeddings & NearestNeighbors model thành công!")
def save_data(features_dict):
    """Lưu embeddings và filenames"""
    filenames = np.array(list(features_dict.keys()))
    feature_list = np.array(list(features_dict.values()))
    
    save_dir = "net3"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Giảm chiều với PCA nếu có đủ dữ liệu
    if len(feature_list) > 5:
        n_components = min(512, len(feature_list))  # Không vượt quá số lượng ảnh
        pca = PCA(n_components=n_components)
        feature_list = pca.fit_transform(feature_list)
        pickle.dump(pca, open(os.path.join(save_dir, "pca.pkl"), "wb"))
        print(f"✅ Áp dụng PCA với {n_components} thành phần!")

    pickle.dump(features_dict, open(os.path.join(save_dir, "features.pkl"), "wb"))
    pickle.dump(filenames, open(os.path.join(save_dir, "filenames.pkl"), "wb"))

    # Tạo mô hình Nearest Neighbors
    nn_model = NearestNeighbors(n_neighbors=4, metric="cosine")
    nn_model.fit(feature_list)
    pickle.dump(nn_model, open(os.path.join(save_dir, "nn_model.pkl"), "wb"))

    print("✅ Đã lưu embeddings & NearestNeighbors model thành công!")

# Hàm tải dữ liệu từ API và trích xuất đặc trưng
def get_image_data(api_url, model):
    """Lấy danh sách ảnh từ API và trích xuất đặc trưng"""
    response = requests.get(api_url)
    data = response.json()

    if response.status_code == 200 and "data" in data:
        images = data["data"]
    else:
        print("❌ Không thể lấy danh sách ảnh từ API!")
        return None

    # Tạo thư mục lưu ảnh nếu chưa tồn tại
    if not os.path.exists("temp"):
        os.makedirs("temp")
    if not os.path.exists("temp/augmented"):
        os.makedirs("temp/augmented")
    if not os.path.exists("temp/original"):
        os.makedirs("temp/original")
    

    features_dict = {}

    for img_info in images:
        img_url = img_info["path"]
        filename = os.path.basename(img_url)

        try:
            # Tải ảnh từ URL
            img_response = requests.get(img_url, stream=True)
            if img_response.status_code == 200:
                img_path = f"temp/original/{filename}"
                with open(img_path, "wb") as f:
                    f.write(img_response.content)

                # Trích xuất đặc trưng từ ảnh gốc
                features = extract_features(img_path, model)
                features_dict[filename] = features

                # Tạo ảnh augmentation và lưu vào thư mục "temp/augmented"
                aug_img = augment_image(img_path)
                aug_img_path = f"temp/augmented/{filename}"
                cv2.imwrite(aug_img_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

                # Trích xuất đặc trưng từ ảnh augmentation
                aug_features = extract_features(aug_img_path, model)
                features_dict[f"{filename}"] = aug_features

            else:
                print(f"⚠️ Không thể đọc ảnh từ {img_url}")

        except Exception as e:
            print(f"❌ Lỗi khi xử lý ảnh {img_url}: {e}")

    return features_dict
# Chạy toàn bộ quy trình
API_URL = "http://localhost:8080/api/v1/public/images"  # Thay bằng URL API của bạn
model = load_model()
features_dict = get_image_data(API_URL, model)
if features_dict:
    save_data(features_dict)

In [2]:

import cv2
import pickle
import numpy as np
import os
import requests
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.applications.efficientnet import EfficientNetB3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from numpy.linalg import norm

def load_model():
    """Tải mô hình EfficientNetB3"""
    base_model = EfficientNetB3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = tf.keras.Sequential([
        base_model,
        GlobalMaxPooling2D()
    ])
    return model

def extract_features(img_path, model):
    """Trích xuất đặc trưng ảnh từ EfficientNetB3"""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)

    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)  # Chuẩn hóa L2
    return normalized_result
# Hàm tạo ảnh mới bằng Data Augmentation
def augment_image(img_path):
    """Tạo ảnh mới bằng Data Augmentation"""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2]
    )

    aug_iter = datagen.flow(img_array, batch_size=1)
    aug_img = next(aug_iter)[0].astype(np.uint8)
    return aug_img
# Hàm tạo ảnh mới bằng Data Augmentation
def augment_image(img_path):
    """Tạo ảnh mới bằng Data Augmentation"""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2]
    )

    aug_iter = datagen.flow(img_array, batch_size=1)
    aug_img = next(aug_iter)[0].astype(np.uint8)
    return aug_img

def send_vector_features(image_id, vector_features, api_url):
    """Gửi vector features lên server"""
    endpoint = f"{api_url}/image/add/vector_feature"
    # Convert numpy array to comma-separated string
    vector_features_str = ','.join(map(str, vector_features.tolist()))
    payload = {
        "id": image_id,
        "vectorFeatures": vector_features_str  # Send as string instead of list
    }
    try:
        response = requests.post(endpoint, json=payload)
        if response.status_code == 200:
            print(f"✅ Đã lưu vector features cho ảnh {image_id}")
            return True
        else:
            print(f"❌ Lỗi khi lưu vector features cho ảnh {image_id}: {response.text}")
            return False
    except Exception as e:
        print(f"❌ Lỗi khi gửi request: {e}")
        return False

def save_data(features_dict):
    """Lưu embeddings và filenames"""
    filenames = np.array(list(features_dict.keys()))
    feature_list = np.array(list(features_dict.values()))
    
    save_dir = "app/net3"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Giảm chiều với PCA nếu có đủ dữ liệu
    if len(feature_list) > 5:
        n_components = min(512, len(feature_list))  # Không vượt quá số lượng ảnh
        pca = PCA(n_components=n_components)
        feature_list = pca.fit_transform(feature_list)
        pickle.dump(pca, open(os.path.join(save_dir, "pca.pkl"), "wb"))
        print(f"✅ Áp dụng PCA với {n_components} thành phần!")

    pickle.dump(features_dict, open(os.path.join(save_dir, "features.pkl"), "wb"))
    pickle.dump(filenames, open(os.path.join(save_dir, "filenames.pkl"), "wb"))

    # Tạo mô hình Nearest Neighbors
    nn_model = NearestNeighbors(n_neighbors=4, metric="cosine")
    nn_model.fit(feature_list)
    pickle.dump(nn_model, open(os.path.join(save_dir, "nn_model.pkl"), "wb"))

    print("✅ Đã lưu embeddings & NearestNeighbors model thành công!")

def process_images(api_url, process_all=False):
    """Xử lý ảnh và trích xuất đặc trưng
    
    Args:
        api_url: Base URL của API (ví dụ: http://localhost:8080/api/v1/public)
        process_all: Nếu True, xử lý tất cả ảnh. Nếu False, chỉ xử lý ảnh chưa có vector features
    """
    # Lấy danh sách ảnh từ API
    response = requests.get(f"{api_url}/images")
    data = response.json()

    if response.status_code != 200 or "data" not in data:
        print("❌ Không thể lấy danh sách ảnh từ API!")
        return None

    images = data["data"]
    total_images = len(images)
    print(f"\n📊 Thống kê ảnh:")
    print(f"   - Tổng số ảnh trong hệ thống: {total_images}")
    
    # Đếm số ảnh đã có vector features (không tính null hoặc empty string)
    images_with_features = sum(1 for img in images if img.get("vectorFeatures") and img.get("vectorFeatures").strip())
    print(f"   - Số ảnh đã có vector features hợp lệ: {images_with_features}")
    print(f"   - Số ảnh cần trích xuất đặc trưng: {total_images - images_with_features if not process_all else total_images}")
    
    # Tạo thư mục data_extract_feature nếu chưa tồn tại
    base_dir = "data_extract_feature"
    original_dir = os.path.join(base_dir, "original")
    augmented_dir = os.path.join(base_dir, "augmented")
    
    for dir_name in [base_dir, original_dir, augmented_dir]:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
            print(f"✅ Đã tạo thư mục {dir_name}")

    features_dict = {}
    processed_count = 0
    failed_count = 0
    model = load_model()
    print("\n🔄 Bắt đầu xử lý ảnh...")

    for idx, img_info in enumerate(images, 1):
        # Bỏ qua ảnh đã có vector features hợp lệ nếu không xử lý tất cả
        if not process_all and img_info.get("vectorFeatures") and img_info.get("vectorFeatures").strip():
            continue

        img_url = img_info["path"]
        image_id = img_info["id"]
        filename = os.path.basename(img_url)
        print(f"\n📝 Đang xử lý ảnh {idx}/{total_images}: {filename}")

        try:
            # Tải ảnh từ URL
            img_response = requests.get(img_url, stream=True)
            if img_response.status_code == 200:
                # Lưu ảnh gốc vào thư mục original
                img_path = os.path.join(original_dir, filename)
                with open(img_path, "wb") as f:
                    f.write(img_response.content)
                print(f"   ✅ Đã lưu ảnh gốc: {filename}")

                # Trích xuất đặc trưng từ ảnh gốc
                features = extract_features(img_path, model)
                print(f"   ✅ Đã trích xuất đặc trưng ({len(features)} chiều)")
                
                # Gửi vector features lên server
                if send_vector_features(image_id, features, api_url):
                    features_dict[filename] = features
                    processed_count += 1
                    print(f"   ✅ Đã lưu vector features lên database")

                # Tạo ảnh augmentation và lưu vào thư mục augmented
                aug_img = augment_image(img_path)
                aug_img_path = os.path.join(augmented_dir, filename)
                cv2.imwrite(aug_img_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))
                print(f"   ✅ Đã tạo và lưu ảnh augmentation")

                # Trích xuất đặc trưng từ ảnh augmentation
                aug_features = extract_features(aug_img_path, model)
                features_dict[f"aug_{filename}"] = aug_features
                print(f"   ✅ Đã trích xuất đặc trưng từ ảnh augmentation")

            else:
                print(f"   ⚠️ Không thể đọc ảnh từ {img_url}")
                failed_count += 1

        except Exception as e:
            print(f"   ❌ Lỗi khi xử lý ảnh {img_url}: {e}")
            failed_count += 1

    print(f"\n📊 Kết quả xử lý:")
    print(f"   - Tổng số ảnh đã xử lý thành công: {processed_count}")
    print(f"   - Số ảnh xử lý thất bại: {failed_count}")
    print(f"   - Tổng số vector features đã lưu (bao gồm cả augmentation): {len(features_dict)}")
    
    if features_dict:
        save_data(features_dict)
        print(f"   - Đã lưu tất cả features vào thư mục app/net3/")
    
    return features_dict

if __name__ == "__main__":
    API_URL = "http://localhost:8080/api/v1/public"  # Base URL của API
    # Chỉ xử lý ảnh chưa có vector features
    process_images(API_URL, process_all=True) 


📊 Thống kê ảnh:
   - Tổng số ảnh trong hệ thống: 50
   - Số ảnh đã có vector features hợp lệ: 50
   - Số ảnh cần trích xuất đặc trưng: 50

🔄 Bắt đầu xử lý ảnh...

📝 Đang xử lý ảnh 1/50: 60b58d41-d6fb-4aa1-8a14-09694fec55a2-ao-so-mi_3.jpg
   ✅ Đã lưu ảnh gốc: 60b58d41-d6fb-4aa1-8a14-09694fec55a2-ao-so-mi_3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
   ✅ Đã trích xuất đặc trưng (1536 chiều)
✅ Đã lưu vector features cho ảnh 01f668f8-ea2b-430e-ab96-92cfa65dc4cc
   ✅ Đã lưu vector features lên database
   ✅ Đã tạo và lưu ảnh augmentation
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
   ✅ Đã trích xuất đặc trưng từ ảnh augmentation

📝 Đang xử lý ảnh 2/50: 33713f51-c95f-42f5-ae92-75cfa931e2fb-ao.webp
   ✅ Đã lưu ảnh gốc: 33713f51-c95f-42f5-ae92-75cfa931e2fb-ao.webp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step
   ✅ Đã trích xuất đặc trưng (1536 chiều)
✅ Đã lưu vector features cho ảnh 064efbdf-3c97-443b-96c4-30e740e6fe43
   ✅ Đã lưu vector features lên database
   ✅ Đã tạo và lưu ảnh augmentation
1/1 ━━━━━━━━